## ovr-svm with weighted bag-of-embeddings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import nltk
import os
import re
import pickle
import sklearn
import sys
import string

from sklearn.metrics import f1_score, precision_score, recall_score,average_precision_score
from sklearn.model_selection import cross_val_score, GridSearchCV,ParameterGrid, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler,MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression,SGDClassifier

from joblib import Parallel, delayed

from tqdm import *

%load_ext autoreload
%autoreload 1

In [2]:
src_dir = os.path.join(os.getcwd(), os.pardir, '../../')
sys.path.append(src_dir)

In [3]:
%aimport src.data.movielens_20m_imdb
%aimport src.helpers.labels,src.helpers.neighbours, src.helpers.segments,src.helpers.embeddings
%aimport src.utils.dataframes, src.utils.clusters, src.utils.metrics,src.utils.plotting

In [4]:
from src.data.movielens_20m_imdb import load_df_or_get_from_cache
from src.helpers.labels import truncate_labels
from src.helpers.segments import make_distance_matrix_for_segments,vectorize_segments
from src.helpers.embeddings import read_glove_wiki_weighted


from src.utils.dataframes import sample_rows
from src.utils.metrics import ranking
from src.utils.plotting import plot_micro_f1_at_k

In [5]:
MODELS_ROOT = os.path.abspath("../../../models/ranking/movielens-ovr-svc-calibrated-embeddings/")
INTERIM_DATA_ROOT = os.path.abspath("../../../data/interim/movielens-ml20m-imdb/")
PATH_TO_PROCESSED_FILE = os.path.abspath('../../../data/processed/movielens-20m-imdb-tags-and-synopses-2017-12-20.csv')

# CONFIGS
SEED= 42

MAX_NB_WORDS = 500

In [6]:
np.random.seed(SEED)

In [7]:
docs_df = load_df_or_get_from_cache(PATH_TO_PROCESSED_FILE,INTERIM_DATA_ROOT)

In [8]:
labels = docs_df["tags"].map(lambda tagstring: tagstring.split(","))

In [9]:
mlb = MultiLabelBinarizer()

binary_labels = mlb.fit_transform(labels)

print("total number of unique tags: {} ".format(len(mlb.classes_)))

data = docs_df['synopsis'].values
indices = np.arange(len(data))

np.random.shuffle(indices)

data = [data[i] for i in indices]
targets = binary_labels[indices]
num_validation_samples = int(0.15 * len(data))

X_train = data[:-num_validation_samples]
Y_train = targets[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = targets[-num_validation_samples:]

print('total number of train documents: {}'.format(len(X_train)))
print('total number of validation documents: {}'.format(len(X_val)))

total number of unique tags: 2138 
total number of train documents: 5704
total number of validation documents: 1006


In [10]:
# only fit on the train data because we're using the IDF scores to weigh the embeddings
vect = TfidfVectorizer(max_features=MAX_NB_WORDS)
vect.fit(X_train)

feature_names = vect.get_feature_names()
idf = vect.idf_
idf_index = dict(zip(vect.get_feature_names(), idf))

## tokenize

In [11]:
# analyzer = preprocess + tokenize
tokenize_func = vect.build_analyzer()

In [12]:
def tokenize(string):
    return tokenize_func(string)

In [13]:
X_train_tok = Parallel(n_jobs=2)(delayed(tokenize)(doc) for doc in X_train)
X_val_tok = Parallel(n_jobs=2)(delayed(tokenize)(doc) for doc in X_val)

## transform into embeddings

In [14]:
embeddings_index = read_glove_wiki_weighted(
    d=100,
    weight_index=idf_index)

overall, 500 out of 500 embeddings were weighted. Total available embeddings: 400000


In [15]:
def build_bag_of_weighted_embeddings(document_tokens):
    out = [embeddings_index[token] for token in document_tokens if token in embeddings_index.keys()]
    
    return np.mean(np.array(out),axis=0)    

In [16]:
X_train_boe = Parallel(n_jobs=2)(delayed(build_bag_of_weighted_embeddings)(tokenized_doc) for tokenized_doc in X_train_tok)
X_train_boe = np.array(X_train_boe)

X_val_boe = Parallel(n_jobs=2)(delayed(build_bag_of_weighted_embeddings)(tokenized_doc) for tokenized_doc in X_val_tok)
X_val_boe = np.array(X_val_boe)

In [17]:
pipeline = Pipeline([
    ('clf', OneVsRestClassifier(CalibratedClassifierCV(SVC(),cv=2),n_jobs=-1)),
])

In [18]:
grid_search_parameters = [
    { 
        "clf__estimator__base_estimator__kernel": ['rbf','linear','poly'],
        "clf__estimator__base_estimator__C": np.logspace(-4,2,7),
        "clf__estimator__base_estimator__degree":[2,3,4,5]
    }
]

In [19]:
parameters = [
    { 
        "clf__estimator__base_estimator__kernel": ['rbf'],
        "clf__estimator__base_estimator__C": [1.0]
    }
]

In [20]:
%%time

for g in tqdm(ParameterGrid(grid_search_parameters)):
    print(g)
    
    pipeline.set_params(**g)
    
    pipeline.fit(X_train_boe,Y_train)
    
    Y_pred_train = pipeline.predict_proba(X_train_boe)    
    Y_pred_val = pipeline.predict_proba(X_val_boe)
    
    ks = [1,2,3,4,5,6,7,8,9,10]

    for k in ks:
        print("train micro-F1 @{}: {}".format(k,ranking.micro_f1_at_k(Y_train,Y_pred_train,k=k,normalize=True)))
        print("validation micro-F1 @{}: {}".format(k,ranking.micro_f1_at_k(Y_val,Y_pred_val,k=k,normalize=True)))    

  0%|          | 0/84 [00:00<?, ?it/s]

{'clf__estimator__base_estimator__C': 0.0001, 'clf__estimator__base_estimator__degree': 2, 'clf__estimator__base_estimator__kernel': 'rbf'}


/home/felipe/tf-venv3/lib/python3.5/site-packages/sklearn/calibration.py:435: RuntimeWarning: overflow encountered in exp
  E = np.exp(AB[0] * F + AB[1])


train micro-F1 @1: 0.2628292980051774
validation micro-F1 @1: 0.2655172413793103
train micro-F1 @2: 0.23814671814671814
validation micro-F1 @2: 0.22887323943661972
train micro-F1 @3: 0.22792937399678972
validation micro-F1 @3: 0.2215674720094284
train micro-F1 @4: 0.21492000156476157
validation micro-F1 @4: 0.20437304774654172
train micro-F1 @5: 0.20927385175650645
validation micro-F1 @5: 0.199391444424557
train micro-F1 @6: 0.20516047828823158
validation micro-F1 @6: 0.19146067415730336
train micro-F1 @7: 0.19774763592046762
validation micro-F1 @7: 0.18458365558133538
train micro-F1 @8: 0.19161035924465492
validation micro-F1 @8: 0.1818592567491246
train micro-F1 @9: 0.18587886069686904
validation micro-F1 @9: 0.18010050251256282


  1%|          | 1/84 [05:58<8:15:14, 358.01s/it]

...

train micro-F1 @10: 0.3435698624965515
validation micro-F1 @10: 0.2139369729249889
{'clf__estimator__base_estimator__C': 100.0, 'clf__estimator__base_estimator__degree': 5, 'clf__estimator__base_estimator__kernel': 'linear'}
train micro-F1 @1: 0.47016226364489744
validation micro-F1 @1: 0.38782051282051283
train micro-F1 @2: 0.41654521479630785
validation micro-F1 @2: 0.3385631709331131
train micro-F1 @3: 0.3762395027755373
validation micro-F1 @3: 0.2982802368198478
train micro-F1 @4: 0.34941763727121466
validation micro-F1 @4: 0.274442538593482
train micro-F1 @5: 0.3273217794199584
validation micro-F1 @5: 0.25861173619525707
train micro-F1 @6: 0.31146853491871623
validation micro-F1 @6: 0.2463683904706566
train micro-F1 @7: 0.2979666652457479
validation micro-F1 @7: 0.23442396891061804
train micro-F1 @8: 0.2854459035488174
validation micro-F1 @8: 0.22672689214498182
train micro-F1 @9: 0.275897969807392
validation micro-F1 @9: 0.2215674720094284


 99%|█████████▉| 83/84 [9:36:53<23:43, 1423.85s/it]

train micro-F1 @10: 0.2661296451098108
validation micro-F1 @10: 0.21361981710023972
{'clf__estimator__base_estimator__C': 100.0, 'clf__estimator__base_estimator__degree': 5, 'clf__estimator__base_estimator__kernel': 'poly'}
train micro-F1 @1: 0.5361221609136404
validation micro-F1 @1: 0.34539473684210525
train micro-F1 @2: 0.4828778509209389
validation micro-F1 @2: 0.30710980227177115
train micro-F1 @3: 0.4312431243124312
validation micro-F1 @3: 0.2685025817555938
train micro-F1 @4: 0.39476553980370777
validation micro-F1 @4: 0.23933493765040473
train micro-F1 @5: 0.3725641245113983
validation micro-F1 @5: 0.22731277533039648
train micro-F1 @6: 0.35386354344532356
validation micro-F1 @6: 0.22130543686459409
train micro-F1 @7: 0.3394468704512373
validation micro-F1 @7: 0.2185681760688085
train micro-F1 @8: 0.32515461268833384
validation micro-F1 @8: 0.21056142301278488
train micro-F1 @9: 0.3113435634282331
validation micro-F1 @9: 0.20321492359595159


100%|██████████| 84/84 [9:44:19<00:00, 1130.62s/it]

train micro-F1 @10: 0.2985070174653601
validation micro-F1 @10: 0.19793999104343932
CPU times: user 6h 10min 36s, sys: 1min 22s, total: 6h 11min 58s
Wall time: 9h 44min 19s


In [24]:
%%time
grid_search_parameters = [
    { 
        "clf__estimator__base_estimator__kernel": ['rbf'],
        "clf__estimator__base_estimator__C":[10,50,100,200,500,1000]
    }
]
for g in tqdm(ParameterGrid(grid_search_parameters)):
    print(g)
    
    pipeline.set_params(**g)
    
    pipeline.fit(X_train_boe,Y_train)
    
    Y_pred_train = pipeline.predict_proba(X_train_boe)    
    Y_pred_val = pipeline.predict_proba(X_val_boe)
    
    ks = [1,2,3,4,5,6,7,8,9,10]

    for k in ks:
        print("train micro-F1 @{}: {}".format(k,ranking.micro_f1_at_k(Y_train,Y_pred_train,k=k,normalize=True)))
        print("validation micro-F1 @{}: {}".format(k,ranking.micro_f1_at_k(Y_val,Y_pred_val,k=k,normalize=True)))    


  0%|          | 0/6 [00:00<?, ?it/s]

{'clf__estimator__base_estimator__C': 10, 'clf__estimator__base_estimator__kernel': 'rbf'}


train micro-F1 @1: 0.6012751348700344
validation micro-F1 @1: 0.3865276663993585
train micro-F1 @2: 0.5408032744947557
validation micro-F1 @2: 0.3289036544850498
train micro-F1 @3: 0.49670561363436705
validation micro-F1 @3: 0.2987598647125141
train micro-F1 @4: 0.45603112840466925
validation micro-F1 @4: 0.2748124330117899
train micro-F1 @5: 0.424875044873388
validation micro-F1 @5: 0.2571032571032571
train micro-F1 @6: 0.4021663009477567
validation micro-F1 @6: 0.2463683904706566
train micro-F1 @7: 0.3825977761124501
validation micro-F1 @7: 0.23773773773773774
train micro-F1 @8: 0.3646202917249041
validation micro-F1 @8: 0.22633608815426998
train micro-F1 @9: 0.3505181139047313
validation micro-F1 @9: 0.21981911128588283


 17%|█▋        | 1/6 [11:27<57:16, 687.39s/it]

train micro-F1 @10: 0.33775316916800235
validation micro-F1 @10: 0.21282643453544148
{'clf__estimator__base_estimator__C': 50, 'clf__estimator__base_estimator__kernel': 'rbf'}
train micro-F1 @1: 0.6125030406227195
validation micro-F1 @1: 0.389111289031225
train micro-F1 @2: 0.5543023697883664
validation micro-F1 @2: 0.3440329218106996
train micro-F1 @3: 0.5032799790081344
validation micro-F1 @3: 0.30354131534569984
train micro-F1 @4: 0.4629479924548639
validation micro-F1 @4: 0.2781343602909713
train micro-F1 @5: 0.43227792436235707
validation micro-F1 @5: 0.2610198789974071
train micro-F1 @6: 0.40911098198721674
validation micro-F1 @6: 0.24585876198779424
train micro-F1 @7: 0.3887655865380735
validation micro-F1 @7: 0.2364137240170298
train micro-F1 @8: 0.37173951828724355
validation micro-F1 @8: 0.2269222295659837
train micro-F1 @9: 0.356985117618819
validation micro-F1 @9: 0.22121807465618862


...

train micro-F1 @10: 0.34337743055051334
validation micro-F1 @10: 0.2150461320085167
{'clf__estimator__base_estimator__C': 500, 'clf__estimator__base_estimator__kernel': 'rbf'}
train micro-F1 @1: 0.6172121212121212
validation micro-F1 @1: 0.3980891719745223
train micro-F1 @2: 0.5618657421998109
validation micro-F1 @2: 0.34743326488706366
train micro-F1 @3: 0.511546992562954
validation micro-F1 @3: 0.3101903695408735
train micro-F1 @4: 0.470367390721373
validation micro-F1 @4: 0.2825437473324797
train micro-F1 @5: 0.43995295790826794
validation micro-F1 @5: 0.26611513271285764
train micro-F1 @6: 0.41669172584487985
validation micro-F1 @6: 0.2547347115801648
train micro-F1 @7: 0.39640146190610065
validation micro-F1 @7: 0.24476570289132601
train micro-F1 @8: 0.37773077128179605
validation micro-F1 @8: 0.2325683540133963
train micro-F1 @9: 0.3615995914850158
validation micro-F1 @9: 0.22278928255962313


 83%|████████▎ | 5/6 [54:28<10:50, 650.84s/it]

train micro-F1 @10: 0.3472654835204636
validation micro-F1 @10: 0.2145709468453279
{'clf__estimator__base_estimator__C': 1000, 'clf__estimator__base_estimator__kernel': 'rbf'}
train micro-F1 @1: 0.6248794599807136
validation micro-F1 @1: 0.39681274900398406
train micro-F1 @2: 0.5691709519628747
validation micro-F1 @2: 0.3501435014350143
train micro-F1 @3: 0.5170292053037525
validation micro-F1 @3: 0.31302403577417554
train micro-F1 @4: 0.4749682507853753
validation micro-F1 @4: 0.28291017708555577
train micro-F1 @5: 0.4450139032768115
validation micro-F1 @5: 0.26521814105880326
train micro-F1 @6: 0.42162062445233595
validation micro-F1 @6: 0.25246091488129707
train micro-F1 @7: 0.40083306632489585
validation micro-F1 @7: 0.24213679480778832
train micro-F1 @8: 0.3814134964972954
validation micro-F1 @8: 0.23081996043086392
train micro-F1 @9: 0.3641056690354036
validation micro-F1 @9: 0.22069372113589467


100%|██████████| 6/6 [1:04:49<00:00, 641.94s/it]

train micro-F1 @10: 0.3499428670610527
validation micro-F1 @10: 0.21567931890741399
CPU times: user 52min 19s, sys: 8.03 s, total: 52min 27s
Wall time: 1h 4min 49s


In [ ]:
ks = [1,2,3,4,5,6,7,8,9,10]

train_scores = [ranking.micro_f1_at_k(Y_train,Y_pred_train,k=k,normalize=True) for k in ks]
validation_scores = [ranking.micro_f1_at_k(Y_val,Y_pred_val,k=k,normalize=True) for k in ks]

In [ ]:
plt.clf()
img = plt.gcf()
ax = plt.gca()
plot_micro_f1_at_k(validation_scores,ax,train_scores)
plt.gcf().set_size_inches(7,5)
plt.show()